# Stable diffusion experiments

## Env. setup

In [ ]:
! nvidia-smi

In [ ]:
! pip install -q diffusers
! pip install -q transformers accelerate
! pip install -q torch torchvision -U

In [ ]:
import os
import torch
import tempfile
from PIL import Image
import matplotlib.pyplot as plt

from diffusers import StableDiffusionImg2ImgPipeline

In [ ]:
torch.__version__

### Global Env. variables

In [ ]:
load_loc = 'templates'
save_loc = 'outputs'

clear_workspace = False

In [ ]:
os.makedirs(load_loc), os.makedirs(save_loc)

### Stable diffusion setup

In [ ]:
model_id_or_path = 'Ojimi/anime-kawai-diffusion'
torch_dtype = torch.float32

# model_id_or_path = "runwayml/stable-diffusion-v1-5"
# torch_dtype=torch.float16

In [ ]:
# Stable diffusion pipe
sd_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype = torch_dtype)
sd_pipe = sd_pipe.to('cuda')

In [ ]:
def generate_temp_name():
  temp = tempfile.NamedTemporaryFile()
  temp_name = temp.name.split('/')[-1]

  return temp_name

In [ ]:
def process(image, pipe, prompt, **kwargs):
  images = pipe(image = image, prompt = prompt, **kwargs).images

  return images

In [ ]:
def load(image_name, image_path = load_loc, shape = (400, 500), format = 'RGB'):
  image_path = os.path.join(image_path, image_name)
  image = Image.open(image_path).convert(format).resize(shape)

  return image

In [ ]:
def save(images, save_loc = save_loc):
  temp_name = generate_temp_name()
  image_loc = os.path.join(save_loc, f'{temp_name}.png')

  images[0].save(image_loc)

  return image_loc

In [ ]:
def plot(images, title):
  plt.title(title)
  plt.imshow(images[0])
  plt.axis('off')
  plt.show()

## Experiments

In [ ]:
image_name = 'img_6.jpg'

input_image = load(image_name)

In [ ]:
prompt = "Upper body, official art, slim, beautiful, aesthetic, looking at viewer, dynamic pose, perfect angle, long hair, petite, beautiful skin, background, beautiful detailed eyes, muted color, depth of field, volumetric lighting, reflection, finest detail, ultra detailed, perfect face, epic proportion, epic composition, professional work"

In [ ]:
negative_prompt = 'lowres, bad anatomy'

In [ ]:
images = process(input_image, sd_pipe, prompt, negative_prompt = negative_prompt, strength=0.75, guidance_scale=3.5)
output_image = save(images)

print(output_image)
plot(images, title = output_image.split('/')[-1])

In [ ]:
if clear_workspace:
  ! rm outputs/*